In [26]:
import json
import csv
import pandas as pd
from collections import Counter

# Cargar el archivo JSON
with open('mobiles.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Asegurar que data es una lista de diccionarios
if isinstance(data, dict):
    data = [data]
elif isinstance(data, list):
    data = [item for item in data if isinstance(item, dict)]
else:
    raise ValueError("El JSON no contiene una estructura válida de datos.")

# Lista de columnas a excluir
columnas_excluidas = {"Numero de pilas de ion de litio", "Fabricante del procesador", "Transcriptor", "Factor de forma", "Resolucion de la camara web frontal", "Color",
    "Contiene liquidos", "Numero de productos", "Pilas / baterias necesarias", "Pilas / baterias incluidas", "Tipo de conexion inalambrica", "Tipo de conectividad", "Que hay en la caja?", "Formatos de audio compatibles", "Otras caracteristicas", "Otras caracteristicas", "Adaptador", "Formatos compatibles", "Tiene estabilizador de imagen",
    "Tipo de medio", "Aparatos compatibles", "Numero de producto", "Nombre del modelo", "Tamano de pantalla", "Entrada de interfaz humana", "Otras funciones de la camara", "Resolucion del sensor optico", "Duracion media de la bateria", "Numero de modelo del producto", "Peso de la bateria de litio", "Peso del producto",
    "Compatible con tecnologia Bluetooth", "Potencia nominal de la bateria", "Pilas","Modelo", "Referencia del fabricante", "Tipo de memoria del ordenador",
    "Hora de espera del telefono (con datos)", "Descripcion de la interfaz de red", "Caracteristicas especiales", "Tecnologia de conectividad", "Resolucion horizontal", "Tecnologia del microfono", "Resolucion vertical max.",
    "Tecnologia de comunicacion inalámbrica", "Duracion media de la pila en stand-by", "Resolucion de captura de video", "Grabador", "Componentes incluidos", "Resolucion efectiva de video",
    "Formatos de audio compatibles", "Series", "Carga electrica de la bateria", "OS", "Tiempo de carga", "Conector de audio",
    "Resolucion de la camara web trasera", "Maxima resolucion", "Pilas incluidas", "Software incluido", "Tamano", "Tipo de conector",
    "Ranuras de memoria disponibles", "Composicion de la pila de la bateria", "Fuente de alimentacion", "Proveedor de conexion inalambrica",
    "Velocidad de transferencia de datos", "Descripcion de la bateria", "Duracion de la bateria media (en horas)",
    "Marca de la tarjeta grafica", "Zoom digital", "Enfoque automatico", "Resolucion del escaner", "Tiempo de carga de la bateria (en horas)",
    "Memoria extraible", "Proveedor de conexion inalambrica", "Tamano de la tarjeta de memoria incluida", "Relacion de aspecto de imagen", "Dimensiones del paquete", "Embalaje de la bateria de litio", "Tipo de dispositivo",
    "Actualizaciones de software garantizadas hasta", "Tecnologia de comunicacion inalambrica", "Compilador", "Numero de pilas de metal de litio", "Voltaje",
    "Resolucion de pantalla", "Tiempo de conversacion telefonica", "Numero de puertos USB", "Contenido de energia de la bateria de litio", "Lector",
    "Marimba", "Tipo de material", "Incluye bateria recargable", "Tiene vision nocturna", "Pantalla a color",
    "Proveedor de conexion inalámbrica", "Tipo de lente", "Descripcion de la tarjeta grafica", "GPS", "Otras caracteristicas de la pantalla",
    "Tipo de conexion inalámbrica", "Numero de puertos"}

# Obtener todas las claves únicas de detalles_tecnicos, excluyendo las no deseadas
all_keys = set()
for item in data:
    if isinstance(item.get('detalles_tecnicos', {}), dict):
        all_keys.update(k for k in item['detalles_tecnicos'].keys() if k not in columnas_excluidas)

# Renombrar y fusionar "RAM" y "Capacidad de la memoria RAM" en "Memoria RAM"
all_keys.discard("RAM")
all_keys.discard("Capacidad de la memoria RAM")
all_keys.add("Memoria RAM")

# Columnas a rellenar con la moda o valores predefinidos
columnas_a_rellenar = {"Tipo de pantalla","Velocidad del procesador", "Tipo de procesador","Frecuencia de actualizacion", "Ano del modelo", "Memoria RAM", "Dimensiones del producto", "Relacion de aspecto", "Tamano de la pantalla", "Tecnologia GSM", "Numero de procesadores"}

# Calcular la moda de cada columna a rellenar
modas = {col: Counter() for col in columnas_a_rellenar}
for item in data:
    detalles = item.get('detalles_tecnicos', {})
    if isinstance(detalles, dict):
        for col in columnas_a_rellenar:
            if col == "Memoria RAM":
                valor = detalles.get("RAM", '') or detalles.get("Capacidad de la memoria RAM", '')
            else:
                valor = detalles.get(col, '')
            if valor:
                modas[col].update([valor])

# Seleccionar el valor más común para cada columna, excepto "Numero de procesadores", que se rellena con "4"
moda_valores = {col: modas[col].most_common(1)[0][0] if modas[col] else ('4' if col == "Numero de procesadores" else '') for col in columnas_a_rellenar}

# Definir los nombres de las columnas
fieldnames = ['product_id', 'precio_actual', 'precio_anterior'] + list(all_keys)

# Abrir un archivo CSV para escritura
with open('mobiles.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Escribir la cabecera
    writer.writeheader()
    
    # Escribir los datos
    for item in data:
        row = {
            'product_id': item.get('product_id', ''),
            'precio_actual': item.get('precio_actual', ''),
            'precio_anterior': item.get('precio_anterior', '')
        }
        detalles = item.get('detalles_tecnicos', {})
        if isinstance(detalles, dict):
            for key in all_keys:
                if key == "Memoria RAM":
                    ram_value = detalles.get("RAM", '') or detalles.get("Capacidad de la memoria RAM", '')
                    row[key] = ram_value if ram_value else moda_valores[key]
                elif key in columnas_a_rellenar:
                    row[key] = detalles.get(key, '') or moda_valores[key]
                else:
                    row[key] = detalles.get(key, '')
        writer.writerow(row)

print("Archivo CSV generado exitosamente: mobiles.csv")


Archivo CSV generado exitosamente: mobiles.csv


In [22]:
datos = pd.read_csv("mobiles.csv")
datos

,product_id,precio_actual,precio_anterior,Tipo de pantalla,Tipo de procesador,Ano del modelo,Memoria RAM,Tecnologia de pantalla,Sistema operativo,Capacidad de la memoria flash instalada,...,Frecuencia de actualizacion,Marca,Capacidad de almacenamiento digital,Capacidad de la memoria,Capacidad de la memoria RAM instalada,Tecnologia GSM,Dimensiones del producto,Modelo,Relacion de aspecto,Tamano de la pantalla
0,B0DR8RHJ9T,"99,.99","159,99",LCD,Snapdragon,2025.0,8 GB,NaN,Android 14,6 GB,...,NaN,OSCAL,2000 GB,64 GB,NaN,4G,"16,7 x 7,7 x 0,79 cm; 189 g",OSCAL FLAT 2C Handy,2.20:1,"6,56 Pulgadas"
1,B0DN5YTHZ9,"149,.99","139,99",LCD,Snapdragon,2024.0,8 GB,NaN,Android 14,NaN,...,NaN,Ulefone,256 GB,2048 GB,12 GB,4G,"16,88 x 7,68 x 0,88 cm; 350 g",Ulefone Note 18 Pro,16:9,"6,78 Pulgadas"
2,B0D4R89TZV,"209,.49","218,98",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B0DFPZXW3P,"79,.99","99,99",LCD,Snapdragon,2024.0,8 GB,NaN,Android 14,64 GB,...,NaN,OUKITEL,64 GB,64 GB,8 GB,4G,"16,7 x 7,7 x 0,79 cm; 189 g",OUKITEL C53,16:9,"6,6 Pulgadas"
4,B0DBLX4FTT,"106,.24","124,99",LCD,Snapdragon,2024.0,8 GB,NaN,Android 14,128 GB,...,90 Hz,UMIDIGI,1024 GB,128 GB,4 GB,4G,"16,7 x 7,7 x 0,79 cm; 189 g",G9T,16:9,"6,75 Pulgadas"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,B0CQJRHSDR,"231,.53","1299,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,B0DG2Y8PXJ,"89,.99","119,99",LCD,Snapdragon,2025.0,8 GB,NaN,Android 14,128 GB,...,60 Hz,OUKITEL,128 GB,128 GB,16 GB,4G,"16,7 x 7,7 x 0,79 cm; 189 g",OUKITEL C57 PRO,16:9,"6,52 Pulgadas"
348,B0C6K9BCXF,"179,.00","999,99",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,B0DGXXTBLJ,"199,.99","219,99",LCD,Snapdragon,2024.0,8 GB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4G,"16,45 x 7,54 x 0,9 cm; 192 g",NaN,16:9,"6,56 Pulgadas"
